In [311]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [19]:
# This requires you to checkout https://github.com/micropython/micropython-lib and avoids the upip
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio/uasyncio/__init__.py lib/uasyncio/
%sendtofile --quiet --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.core/uasyncio/core.py lib/uasyncio/
%sendtofile --quiet --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.queues/uasyncio/queues.py lib/uasyncio/
%sendtofile --quiet --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.synchro/uasyncio/synchro.py lib/uasyncio/

# This requires you to checkout https://github.com/peterhinch/micropython-async
%sendtofile --quiet --source /home/julian/extrepositories/micropython-async/asyn.py
%sendtofile --quiet --source /home/julian/extrepositories/micropython-async/aswitch.py

# This requires you to checkout https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as
%sendtofile --quiet --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py


Sent 258 lines (8400 bytes) to lib/uasyncio/__init__.py.
Sent 315 lines (9509 bytes) to lib/uasyncio/core.py.
Sent 94 lines (2691 bytes) to lib/uasyncio/queues.py.
Sent 28 lines (846 bytes) to lib/uasyncio/synchro.py.
Sent 470 lines (14738 bytes) to asyn.py.
Sent 219 lines (8811 bytes) to aswitch.py.
Sent 638 lines (23042 bytes) to mqtt_as.py.


In [404]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [405]:
%sendtofile ledstrippanel.py

import framebuf
from machine import Pin, SPI

cs, spi = None, None
brightnesschars = None
buffer = None
fbuff = None
numledchars = 0
# ESP32-HSPI: sck=14, mosi=13

def setbrightness(brightness, i0=0, i1=9999):
    # 0<=brightness<=15
    for i in range(max(i0, 0), min(i1, numledchars)):
        brightnesschars[(numledchars - i)*2 - 1] = brightness
        
def showbrightness():
    cs.value(0)  
    spi.write(brightnesschars)
    cs.value(1)
    
s = bytearray(8)
def show():
    for y in range(8):
        s[6] = s[4] = s[2] = s[0] = y+1
        yp = y*(numledchars+1) + numledchars
        cs.value(0)  # cannot put in loop as resets the shifting
        for m in range(0, numledchars, 4):
            s[1] = buffer[yp - m - 1]
            s[3] = buffer[yp - m - 2]
            s[5] = buffer[yp - m - 3]
            s[7] = buffer[yp - m - 4]
            spi.write(s)
        cs.value(1)

def setup(lnumledchars, cspin=26, sckpin=14, mosipin=13):
    global numledchars, cs, spi, brightnesschars, buffer, fbuff
    numledchars = lnumledchars
    spi = SPI(1, 1000000, sck=Pin(sckpin), mosi=Pin(mosipin))  
    cs = Pin(cspin, Pin.OUT)

    for i in range(0, 10, 2):
        cs.value(0)
        x = b"\x0c\x00\x0f\x00\x0b\x07\x09\x00\x0c\x01"[i:i+2]
        spi.write(x*numledchars)
        cs.value(1)
    brightnesschars = bytearray(bytes([0x0a, 0])*numledchars)
    setbrightness(0)
    showbrightness()

    buffer = bytearray(8*(numledchars+1))  # make one extra character to scroll from
    fbuffwidth = 8*(numledchars+1)
    fbuffheight = 8
    fbuff = framebuf.FrameBuffer(buffer, fbuffwidth, fbuffheight, framebuf.MONO_HLSB)

    for i in range(0, numledchars*8, 4):
        fbuff.fill_rect(i, (i%8), 4, 4, 1)
    show()
    return fbuff


Sent 58 lines (1751 bytes) to ledstrippanel.py.


In [406]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.100.1
pinled       2
boardname    esp32ledstrip2
numledchars  60
framedelay   5

Sent 7 lines (148 bytes) to config.txt.


In [345]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [346]:
#%sendtofile main.py

fconfig = dict(x.split()  for x in open("config.txt"))
numledchars = int(fconfig.get("numledchars", 20))
framedelay = int(fconfig.get("framedelay", 10))

from ledstrippanel import setup, show, setbrightness, showbrightness
fbuff = setup(numledchars)


In [347]:
#%sendtofile --append main.py

import time

textmessage = ""
headerlength = 0
i = 0

def setmessage(ltextmessage):
    global textmessage, headerlength, i
    textmessage = ltextmessage
    headerlength = max(textmessage.find(" "), 0)
    setbrightness(0, i1=9999)
    showbrightness()
    i = 0

async def ascrollnextchar():
    global i
    if i < len(textmessage) + numledchars - headerlength:
        fbuff.fill_rect(numledchars*8, 0, 8, 8, 0)
        c = textmessage[i]  if i < len(textmessage) else " "
        fbuff.text(c, numledchars*8, 0, 1)
        if headerlength:
            if i < numledchars:
                setbrightness(3, numledchars-i-1, numledchars-i)
                setbrightness(0, numledchars-i+headerlength, numledchars-i+headerlength+1)
                showbrightness()
            elif i == numledchars:
                setbrightness(0, headerlength, headerlength+1)
                showbrightness()

        for j in range(8):
            fbuff.scroll(-1, 0)
            if i >= numledchars:
                fbuff.fill_rect(0, 0, headerlength*8, 8, 0)
                fbuff.text(textmessage[:headerlength], 0, 0, 1)
            show()
            #time.sleep_ms(5)
            await asyncio.sleep_ms(1)
    i += 1

#setmessage("ffof777 short one")
#for k in range(50):
#    scrollnextchar()
        

In [318]:
setmessage("#88 hihi annas a a a a a a a a affffff gggg")
for i in range(200):
    scrollnextchar()



...

In [ ]:
aloop = asyncio.get_event_loop()
aloop.create_task(shownexttask())


In [337]:
%ls lib

Listing directory 'lib'.
             lib/uasyncio/


In [348]:
#%sendtofile --append main.py

from mqtt_as import MQTTClient, config
import uasyncio as asyncio
from machine import Pin
import ubinascii

config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['server'] = fconfig["mqttbroker"]
pled = Pin(int(fconfig["pinled"]), Pin.OUT)
print(config)
newmessage = None

def callback(topic, msg):
    global newmessage
    print("Dcallback", (topic, msg))
    newmessage = msg.decode()

async def IPnumber_task(client):
    topicip = fconfig["boardname"]+"/ip"
    topicmac = fconfig["boardname"]+"/mac"
    ipnumber = client._sta_if.ifconfig()[0]
    macaddress = ubinascii.hexlify(client._sta_if.config('mac'),':').decode().upper()
    while True:
        await client.publish(topicip, ipnumber, retain=True)
        await client.publish(topicmac, macaddress, retain=True)
        await asyncio.sleep_ms(20000)
        
config['subs_cb'] = callback
MQTTClient.DEBUG = True
client = MQTTClient(config)

aloop = asyncio.get_event_loop()

async def shownexttask():
    global newmessage
    topicready = fconfig["boardname"]+"/ready"
    while True:
        if i == len(textmessage) and newmessage is None:
            await client.publish(topicready, "next", retain=True)
        if newmessage is not None and i > len(textmessage) + 3:
            setmessage(newmessage)
            newmessage = None
        await ascrollnextchar()

aloop.create_task(shownexttask())

newmessage = "Connecting"
aloop.run_until_complete(client.connect())
aloop.create_task(IPnumber_task(client))

topicmessage = fconfig["boardname"]+"/message"
newmessage = "Connected"
aloop.run_until_complete(client.subscribe(topicmessage, 1))
newmessage = "Subscribed"

aloop.run_forever()


{'wifi_coro': <generator>, 'server': '10.0.100.1', 'ssl': False, 'port': 0, 'ssl_params': {}, 'response_time': 10, 'will': None, 'subs_cb': <function <lambda> at 0x3ffc4680>, 'ssid': 'DoESLiverpool', 'wifi_pw': 'decafbad00', 'clean_init': True, 'user': '', 'max_repubs': 4, 'password': '', 'ping_interval': 0, 'connect_coro': <generator>, 'clean': True, 'keepalive': 60, 'client_id': b'cc50e3961448'}
Checking WiFi integrity.
.Got reliable connection
Connecting to broker.
Connected to broker.
.Dcallback (b'esp32ledstrip2/message', b'#1184 Consider blinds for the East wall of Room 29')
Dcallback (b'esp32ledstrip2/message', b'#1213 Need to backup MQTT server database')
..Dcallback (b'esp32ledstrip2/message', b'#1179 Make a note of all the settings in Sophia and Gerald')
.RAM free 80672 alloc 37664
..Dcallback (b'esp32ledstrip2/message', b'#1215 Install some storage for ear plugs in the CNC router')
..RAM free 80672 alloc 37664
.Dcallback (b'esp32ledstrip2/message', b'#1181 Cut Worktop in Alc

Traceback (most recent call last):
  File "<stdin>", line 58, in <module>
  File "/lib/uasyncio/core.py", line 154, in run_forever
  File "/lib/uasyncio/core.py", line 109, in run_forever
  File "<stdin>", line 45, in shownexttask
  File "<stdin>", line 37, in ascrollnextchar
  File "ledstrippanel.py", line 32, in show
KeyboardInterrupt: 


In [352]:
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/umqtt.robust/umqtt/robust.py lib/umqtt/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/umqtt.simple/umqtt/simple.py lib/umqtt/


Sent 204 lines (6411 bytes) to lib/umqtt/simple.py.


In [160]:
%sendtofile main.py

from ledgridpanel import fbuff, show, fbuffwidth, fbuffheight, numledcells
fwidth = fbuffwidth - 8  # the last section is the scroll buffer
fledchars = numledcells

import time

newmessage = None
message = ""
prevmessage = ""
ys = 0
xchar = 0
def shownext(i):
    global newmessage, message, prevmessage, ys, xchar
    if newmessage is not None and ys == 0:
        prevmessage = message
        message = newmessage
        newmessage = None
        xchar = 0
        for c1, c2 in zip(prevmessage, message):
            if c1 != c2:
                break
            xchar += 1
        ys = 8
        fbuff.fill(0)
        fbuff.text(message, 0, 0, 1)
    if ys != 0:
        ys -= 1
        fbuff.fill_rect(xchar*8, 0, (fledchars - xchar)*8, 8, 0)
        fbuff.text(prevmessage[xchar:], xchar*8, ys - 8, 1)
        fbuff.text(message[xchar:], xchar*8, ys, 1)
        show()
    else:
        if (i % 10) == 0:
            fbuff.fill_rect((fledchars-1)*8, 7, 8, 1, (i//10)%2)
            show()




from mqtt_as import MQTTClient, config
import uasyncio as asyncio
from machine import Pin
import ubinascii

fconfig = dict(x.split()  for x in open("config.txt"))
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['server'] = fconfig["mqttbroker"]
pled = Pin(int(fconfig["pinled"]), Pin.OUT)
print(config)


def callback(topic, msg):
    global newmessage
    print("Dcallback", (topic, msg))
    newmessage = msg.decode()

async def IPnumber_task(client):
    topicip = fconfig["boardname"]+"/ip"
    topicmac = fconfig["boardname"]+"/mac"
    ipnumber = client._sta_if.ifconfig()[0]
    macaddress = ubinascii.hexlify(client._sta_if.config('mac'),':').decode().upper()
    while True:
        await client.publish(topicip, ipnumber, retain=True)
        await client.publish(topicmac, macaddress, retain=True)
        await asyncio.sleep_ms(20000)
        
config['subs_cb'] = callback
MQTTClient.DEBUG = True
client = MQTTClient(config)

aloop = asyncio.get_event_loop()

async def shownexttask():
    i = 0
    while True:
        shownext(i)
        i += 1
        await asyncio.sleep_ms(50)

aloop.create_task(shownexttask())

newmessage = "Connecting"
aloop.run_until_complete(client.connect())
aloop.create_task(IPnumber_task(client))

topicmessage = fconfig["boardname"]+"/message"
newmessage = "Connected"
aloop.run_until_complete(client.subscribe(topicmessage, 1))
newmessage = "Subscribed"

aloop.run_forever()


Sent 92 lines (2462 bytes) to main.py.


In [6]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
ets Jun  8 2016 00:22:57

rst:0x1 (POWERON_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0018,len:4
load:0x3fff001c,len:4928
ho 0 tail 12 room 4
load:0x40078000,len:9404
load:0x40080400,len:6228
entry 0x400806ec
MicroPython v1.11-240-g519746cae on 2019-08-26; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


In [11]:
x = bytearray([0x0a, 1]*8)
x[3] = 255
print(x)


bytearray(b'\n\x01\n\xff\n\x01\n\x01\n\x01\n\x01\n\x01\n\x01')


In [18]:
fbuff.fill(0)
fbuff.text("hi there; "*13, 0, 0, 1)
show()

In [139]:
aloop.run_forever()


Dcallback (b'esp32ledstrip/message', b'Power: 2.868kWh')
Dcallback (b'esp32ledstrip/message', b'Power: 2.869kWh')
Dcallback (b'esp32ledstrip/message', b'Power: 2.87kWh')
Dcallback (b'esp32ledstrip/message', b'Power: 2.871kWh')
Dcallback (b'esp32ledstrip/message', b'Power: 2.872kWh')
Dcallback (b'esp32ledstrip/message', b'Power: 2.873kWh')
Dcallback (b'esp32ledstrip/message', b'Power: 2.874kWh')
Dcallback (b'esp32ledstrip/message', b'Power: 2.875kWh')
Dcallback (b'esp32ledstrip/message', b'Power: 2.876kWh')
Dcallback (b'esp32ledstrip/message', b'Power: 2.877kWh')
Dcallback (b'esp32ledstrip/message', b'Power: 2.878kWh')
Dcallback (b'esp32ledstrip/message', b'Power: 2.879kWh')
Dcallback (b'esp32ledstrip/message', b'Power: 2.88kWh')
Dcallback (b'esp32ledstrip/message', b'Power: 2.881kWh')
Dcallback (b'esp32ledstrip/message', b'Power: 2.882kWh')
Dcallback (b'esp32ledstrip/message', b'Power: 2.883kWh')
Dcallback (b'esp32ledstrip/message', b'Power: 2.884kWh')
Dcallback (b'esp32ledstrip/messag

Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
  File "/lib/uasyncio/core.py", line 173, in run_forever
  File "/lib/uasyncio/__init__.py", line 69, in wait
KeyboardInterrupt: 


In [65]:
help(client)

object <MQTTClient object at 3ffc7850> is of type MQTTClient
  DEBUG -- True
  _keep_alive -- <generator>
  _reconnect -- <function _reconnect at 0x3ffc67a0>
  subscribe -- <closure>
  __qualname__ -- MQTTClient
  __init__ -- <closure>
  wifi_connect -- <generator>
  connect -- <generator>
  isconnected -- <function isconnected at 0x3ffc6730>
  _connection -- <generator>
  _keep_connected -- <generator>
  publish -- <closure>
  __module__ -- mqtt_as
  _memory -- <generator>
  _handle_msg -- <generator>


In [51]:
async def shownexttask():
    i = 0
    while True:
        shownext(i)
        i += 1
        await asyncio.sleep_ms(50)

aloop.create_task(shownexttask())
#aloop.run_until_complete(plotdevices(fqm))



fconfig["boardname"]



Reconnect: broker fail.
.

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 10, in <module>
  File "/lib/uasyncio/core.py", line 173, in run_forever
  File "/lib/uasyncio/__init__.py", line 69, in wait
KeyboardInterrupt: 


In [6]:
import time
message = "DoESLiverpool is... a long time coming;;;"
messagechar = -1
messagepix = 7

def msgnext():
    global messagechar, messagepix
    messagepix += 1
    if messagepix < 8:
        fbuff.scroll(-1,0)
    else:
        messagechar += 1
        messagepix = 0
        fbuff.fill(0)
        mc = max(messagechar, 0)
        for y, message in enumerate(messages):
            fbuff.text(message[mc:mc+numledcells+1], -min(0, messagechar)*8, 0, 1)
    show()

while 1:
    msgnext()
    if messagechar >= len(messages[0]):
        messagechar, messagepix = -numledcells, 7
    #time.sleep_ms(20)



......

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 22, in <module>
  File "<stdin>", line 19, in msgnext
  File "<stdin>", line 41, in show
KeyboardInterrupt: 


In [401]:
# non-async kind
%sendtofile main.py

fconfig = dict(x.split()  for x in open("config.txt"))
numledchars = int(fconfig.get("numledchars", 20))
framedelay = int(fconfig.get("framedelay", 10))

from ledstrippanel import setup, show, setbrightness, showbrightness
fbuff = setup(numledchars)

import time

textmessage = ""
headerlength = 0
i = 0

def setmessage(ltextmessage):
    global textmessage, headerlength, i
    textmessage = ltextmessage
    headerlength = max(textmessage.find(" "), 0)
    setbrightness(0, i1=9999)
    showbrightness()
    i = 0

def scrollnextchar():
    global i
    if i < len(textmessage) + numledchars - headerlength:
        fbuff.fill_rect(numledchars*8, 0, 8, 8, 0)
        c = textmessage[i]  if i < len(textmessage) else " "
        fbuff.text(c, numledchars*8, 0, 1)
        if headerlength:
            if i < numledchars:
                setbrightness(3, numledchars-i-1, numledchars-i)
                setbrightness(0, numledchars-i+headerlength, numledchars-i+headerlength+1)
                showbrightness()
            elif i == numledchars:
                setbrightness(0, headerlength, headerlength+1)
                showbrightness()

        for j in range(8):
            fbuff.scroll(-1, 0)
            if i >= numledchars:
                fbuff.fill_rect(0, 0, headerlength*8, 8, 0)
                fbuff.text(textmessage[:headerlength], 0, 0, 1)
            show()
            #time.sleep_ms(5)
            time.sleep_ms(1)
    i += 1

#setmessage("ffof777 short one")
#for k in range(50):
#    scrollnextchar()
        

Sent 50 lines (1533 bytes) to main.py.


In [402]:
%sendtofile --append main.py

import machine, time, network


wifiname = fconfig["wifiname"]
wifipassword = fconfig["wifipassword"]
mqttbroker = fconfig["mqttbroker"]
pled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)

pled.value(1)
time.sleep(3.5)  # chance to interrupt before wifi is turned on

si = network.WLAN(network.STA_IF)
si.active(True)

print("Connecting to", wifiname)
si.connect(wifiname, wifipassword)
while not si.isconnected():
    time.sleep(0.1)
pled.value(0) 
print("connected to wifi")

for i in range(10):
    pled.value(i%2)
    time.sleep_ms(100)
pled.value(1) 

boardname =  fconfig["boardname"]

from umqtt.robust import MQTTClient
client = MQTTClient(boardname, mqttbroker, 1883)
for i in range(100):
    try:
        client.connect()
        break
    except OSError as e:
        print(e)
pled.value(0) 
print("connected to mqtt")

for i in range(10):
    pled.value(i%2)
    time.sleep_ms(50)

topicmessage = boardname+"/message"
topicready = boardname+"/ready"
client.publish(boardname+"/ip", si.ifconfig()[0], retain=True, qos=1)

newmessage = None

def sub_cb(topic, msg):
    print((topic, msg))
    global newmessage
    print("Dcallback", (topic, msg))
    newmessage = msg.decode()

client.set_callback(sub_cb)
client.subscribe(topicmessage)

#while True:
#    client.wait_msg()

    


Sent 62 lines (1301 bytes) to main.py.


In [403]:
%sendtofile --append main.py

while True:
    if i == len(textmessage) and newmessage is None:
        client.publish(topicready, "next")
    if newmessage is not None and i > len(textmessage) + 3:
        setmessage(newmessage)
        newmessage = None
    scrollnextchar()
    client.check_msg()
    time.sleep_ms(1)



Sent 10 lines (291 bytes) to main.py.


In [397]:
print(i, textmessage)

26 #1184 Consider blinds for the East wall of Room 29


In [398]:
fbuff.fill(1)
show()